In [82]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization

from keras.utils import np_utils

In [83]:
df = pd.read_csv('data/VTOUTP16.TXT', low_memory=False)
df.head()

,hnum2,ATYPE,asour,intage,TXTZIP,sex,dstat,PPAY,CHRGS,DX1,...,BTYPE,ERFLAG,cah,vtres,OBSFLAG,AFLAG,Uniq,ADMID_QTR,DISCD_QTR,CHRGS_HCIA
0,11,3,4,13,050,2,5,1,3409.85,L600,...,131,0,1,1,0,1,3,1,1,3409.85
1,11,3,4,11,050,2,5,7,1837.65,Z86010,...,131,0,1,1,0,1,54,1,1,1837.65
2,11,3,4,12,037,2,5,1,1102.65,Z1211,...,131,0,1,3,0,1,85,1,1,1102.65
3,11,3,4,10,050,1,5,7,1102.70,Z1211,...,131,0,1,1,0,1,87,1,1,1102.70
4,11,3,4,13,051,2,5,7,1837.65,Z1211,...,131,0,1,1,0,1,93,1,1,1837.65


In [84]:
# clean data
## only use the procedure columns
df.drop(df.columns[0:29],axis=1,inplace=True) # delete everything before PX columns
df.drop(df.columns[11:],axis=1,inplace=True) # delete everything after the first 10 PX columns (the last 10 PX columns are empty)

## delete columns where no procedurestook place
df = df[df.PX1 != ' ']
df.head()

,PX1,PX2,PX3,PX4,PX5,PX6,PX7,PX8,PX9,PX10,PX11
535,0HRV37Z,0UT2FZZ,0JDM3ZZ,0JDL3ZZ,0WQF0ZZ,0UT7FZZ,3E1P38Z,,,,
6226,09Q10ZZ,0JQ00ZZ,0HQ1XZZ,0CQ0XZZ,,,,,,,
8661,B246ZZZ,,,,,,,,,,
12438,30233N1,,,,,,,,,,
13316,0DB68ZX,0D748ZZ,,,,,,,,,


In [85]:
# clean up data
## replace spaces with 0
df.replace([' '], [0], inplace=True)

In [86]:
## replace all ICD-9 codes with ints in the PX columns
icd9codes_dict = {}
replacement = 0
for i in range (1,12):
    current_column = f'PX{i}'
    for icd9code in df[current_column]:
        if icd9code not in icd9codes_dict.keys():
            icd9codes_dict[icd9code] = replacement
            replacement += 1

for i in range (1,12):
    current_column = f'PX{i}'
    df[current_column] = df[current_column].map(icd9codes_dict)
    df[current_column] = pd.to_numeric(df[current_column])

In [87]:
# set features and targets
y = [] ## y = next procedure
X = [] ## X = current procedure
for row in df.values:
    for i in range(0, len(row)):
        if(row[i] == icd9codes_dict[0]): # if value is empty (equivalent of zero)
            y.append(row[i-1]) # y is the value of the column before the one with the zero value
            X.append(row[i-2]) # X is the value of the column before y
            break # go to next row

In [92]:
# check first row
print("full row:\n{0}\n".format(df.iloc[0]))
print("X: {0}".format(X[0]))
print("y: {0}".format(y[0]))

full row:
PX1        0
PX2     1347
PX3     1691
PX4     1813
PX5       12
PX6     1417
PX7     1896
PX8     1348
PX9     1348
PX10    1348
PX11    1348
Name: 535, dtype: int64

X: 1417
y: 1896


In [93]:
X = np.asarray(X)
y = np.asarray(y)

In [94]:
# split into training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [95]:
print(X_train.shape)

(16442,)


In [96]:
# One-hot encoding
y_train = np_utils.to_categorical(y_train)

In [97]:
print(y_train)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [125]:
# setup model
## parameters
model = Sequential()
dropoutrate = 0.2
batchsize = 1024
epochs = 1000
inputdim = 1
adam = keras.optimizers.Adam() # Adam optimizer

model.add(Dense(100, input_dim=inputdim, kernel_initializer='uniform',activation='relu'))
model.add(Dense(100, kernel_initializer='uniform',activation='relu'))
model.add(Dense(1911, activation='softmax'))

In [126]:
model.compile(loss=keras.losses.categorical_crossentropy,optimizer=adam,metrics=['accuracy'])

In [127]:
history = model.fit(X_train, y_train, epochs=epochs,validation_split=0.3, batch_size=batchsize,verbose=1)

Train on 11509 samples, validate on 4933 samples
Epoch 1/1000
11509/11509 [==============================] - 9s 763us/step - loss: 7.1419 - acc: 0.0319 - val_loss: 6.4854 - val_acc: 0.0701
Epoch 2/1000
11509/11509 [==============================] - 5s 455us/step - loss: 6.3019 - acc: 0.0591 - val_loss: 6.2466 - val_acc: 0.0762
Epoch 3/1000
11509/11509 [==============================] - 5s 447us/step - loss: 6.1527 - acc: 0.0735 - val_loss: 6.2497 - val_acc: 0.0669
Epoch 4/1000
11509/11509 [==============================] - 5s 461us/step - loss: 6.1105 - acc: 0.0738 - val_loss: 6.2589 - val_acc: 0.0693
Epoch 5/1000
11509/11509 [==============================] - 5s 447us/step - loss: 6.0859 - acc: 0.0726 - val_loss: 6.2566 - val_acc: 0.0780
Epoch 6/1000
11509/11509 [==============================] - 5s 449us/step - loss: 6.0637 - acc: 0.0726 - val_loss: 6.2633 - val_acc: 0.0774
Epoch 7/1000
11509/11509 [==============================] - 5s 452us/step - loss: 6.0442 - acc: 0.0708 - val_lo

Epoch 59/1000
11509/11509 [==============================] - 5s 455us/step - loss: 4.6848 - acc: 0.0990 - val_loss: 5.0993 - val_acc: 0.1001
Epoch 60/1000
11509/11509 [==============================] - 5s 450us/step - loss: 4.6806 - acc: 0.0989 - val_loss: 5.1025 - val_acc: 0.1001
Epoch 61/1000
11509/11509 [==============================] - 5s 455us/step - loss: 4.6750 - acc: 0.0990 - val_loss: 5.1002 - val_acc: 0.1001
Epoch 62/1000
11509/11509 [==============================] - 6s 497us/step - loss: 4.6723 - acc: 0.0990 - val_loss: 5.0937 - val_acc: 0.1001
Epoch 63/1000
11509/11509 [==============================] - 6s 493us/step - loss: 4.6712 - acc: 0.0990 - val_loss: 5.0979 - val_acc: 0.1003
Epoch 64/1000
11509/11509 [==============================] - 5s 451us/step - loss: 4.6675 - acc: 0.0991 - val_loss: 5.0914 - val_acc: 0.1001
Epoch 65/1000
11509/11509 [==============================] - 5s 450us/step - loss: 4.6659 - acc: 0.0994 - val_loss: 5.0930 - val_acc: 0.1003
Epoch 66/1000

Epoch 117/1000
11509/11509 [==============================] - 5s 456us/step - loss: 4.5702 - acc: 0.1117 - val_loss: 5.0522 - val_acc: 0.1101
Epoch 118/1000
11509/11509 [==============================] - 5s 453us/step - loss: 4.5678 - acc: 0.1075 - val_loss: 5.0555 - val_acc: 0.1012
Epoch 119/1000
11509/11509 [==============================] - 5s 452us/step - loss: 4.5754 - acc: 0.1038 - val_loss: 5.0812 - val_acc: 0.1003
Epoch 120/1000
11509/11509 [==============================] - 5s 452us/step - loss: 4.5819 - acc: 0.1037 - val_loss: 5.0534 - val_acc: 0.1010
Epoch 121/1000
11509/11509 [==============================] - 5s 450us/step - loss: 4.5712 - acc: 0.1050 - val_loss: 5.0649 - val_acc: 0.1143
Epoch 122/1000
11509/11509 [==============================] - 5s 455us/step - loss: 4.5762 - acc: 0.1044 - val_loss: 5.0501 - val_acc: 0.1083
Epoch 123/1000
11509/11509 [==============================] - 5s 450us/step - loss: 4.5616 - acc: 0.1066 - val_loss: 5.0489 - val_acc: 0.1095
Epoch 

Epoch 175/1000
11509/11509 [==============================] - 5s 450us/step - loss: 4.5467 - acc: 0.1037 - val_loss: 5.0628 - val_acc: 0.1091
Epoch 176/1000
11509/11509 [==============================] - 5s 454us/step - loss: 4.5330 - acc: 0.1039 - val_loss: 5.0344 - val_acc: 0.1133
Epoch 177/1000
11509/11509 [==============================] - 5s 453us/step - loss: 4.5240 - acc: 0.1074 - val_loss: 5.0318 - val_acc: 0.1233
Epoch 178/1000
11509/11509 [==============================] - 5s 448us/step - loss: 4.5111 - acc: 0.1093 - val_loss: 5.0335 - val_acc: 0.1095
Epoch 179/1000
11509/11509 [==============================] - 5s 458us/step - loss: 4.5041 - acc: 0.1088 - val_loss: 5.0392 - val_acc: 0.1113
Epoch 180/1000
11509/11509 [==============================] - 5s 449us/step - loss: 4.5100 - acc: 0.1097 - val_loss: 5.0376 - val_acc: 0.1097
Epoch 181/1000
11509/11509 [==============================] - 5s 455us/step - loss: 4.5013 - acc: 0.1115 - val_loss: 5.0576 - val_acc: 0.1101
Epoch 

Epoch 233/1000
11509/11509 [==============================] - 5s 452us/step - loss: 4.4596 - acc: 0.1210 - val_loss: 5.0445 - val_acc: 0.1174
Epoch 234/1000
11509/11509 [==============================] - 5s 452us/step - loss: 4.4655 - acc: 0.1168 - val_loss: 5.0325 - val_acc: 0.1360
Epoch 235/1000
11509/11509 [==============================] - 5s 453us/step - loss: 4.4619 - acc: 0.1155 - val_loss: 5.0332 - val_acc: 0.1340
Epoch 236/1000
11509/11509 [==============================] - 5s 451us/step - loss: 4.4622 - acc: 0.1203 - val_loss: 5.0348 - val_acc: 0.1194
Epoch 237/1000
11509/11509 [==============================] - 5s 457us/step - loss: 4.4627 - acc: 0.1140 - val_loss: 5.0404 - val_acc: 0.1137
Epoch 238/1000
11509/11509 [==============================] - 5s 453us/step - loss: 4.4579 - acc: 0.1220 - val_loss: 5.0358 - val_acc: 0.1164
Epoch 239/1000
11509/11509 [==============================] - 5s 456us/step - loss: 4.4544 - acc: 0.1162 - val_loss: 5.0376 - val_acc: 0.1135
Epoch 

...

Epoch 755/1000
11509/11509 [==============================] - 5s 476us/step - loss: 4.2904 - acc: 0.1401 - val_loss: 4.9907 - val_acc: 0.1380
Epoch 756/1000
11509/11509 [==============================] - 5s 451us/step - loss: 4.2457 - acc: 0.1444 - val_loss: 4.9954 - val_acc: 0.1417
Epoch 757/1000
11509/11509 [==============================] - 5s 452us/step - loss: 4.2425 - acc: 0.1481 - val_loss: 4.9945 - val_acc: 0.1492
Epoch 758/1000
11509/11509 [==============================] - 5s 459us/step - loss: 4.2413 - acc: 0.1468 - val_loss: 4.9918 - val_acc: 0.1389
Epoch 759/1000
11509/11509 [==============================] - 5s 456us/step - loss: 4.2365 - acc: 0.1470 - val_loss: 4.9939 - val_acc: 0.1419
Epoch 760/1000
11509/11509 [==============================] - 5s 450us/step - loss: 4.2394 - acc: 0.1422 - val_loss: 4.9972 - val_acc: 0.1348
Epoch 761/1000
11509/11509 [==============================] - 5s 449us/step - loss: 4.2456 - acc: 0.1454 - val_loss: 4.9962 - val_acc: 0.1441
Epoch 

Epoch 813/1000
11509/11509 [==============================] - 5s 457us/step - loss: 4.2231 - acc: 0.1497 - val_loss: 4.9853 - val_acc: 0.1350
Epoch 814/1000
11509/11509 [==============================] - 5s 453us/step - loss: 4.2250 - acc: 0.1475 - val_loss: 4.9864 - val_acc: 0.1464
Epoch 815/1000
11509/11509 [==============================] - 5s 451us/step - loss: 4.2236 - acc: 0.1501 - val_loss: 5.0044 - val_acc: 0.1366
Epoch 816/1000
11509/11509 [==============================] - 5s 454us/step - loss: 4.2730 - acc: 0.1456 - val_loss: 5.0289 - val_acc: 0.1364
Epoch 817/1000
11509/11509 [==============================] - 5s 451us/step - loss: 4.2479 - acc: 0.1421 - val_loss: 4.9812 - val_acc: 0.1433
Epoch 818/1000
11509/11509 [==============================] - 5s 451us/step - loss: 4.2260 - acc: 0.1474 - val_loss: 4.9907 - val_acc: 0.1466
Epoch 819/1000
11509/11509 [==============================] - 5s 451us/step - loss: 4.2293 - acc: 0.1459 - val_loss: 4.9866 - val_acc: 0.1405
Epoch 

Epoch 871/1000
11509/11509 [==============================] - 5s 447us/step - loss: 4.2226 - acc: 0.1443 - val_loss: 4.9952 - val_acc: 0.1401
Epoch 872/1000
11509/11509 [==============================] - 5s 456us/step - loss: 4.2135 - acc: 0.1472 - val_loss: 4.9807 - val_acc: 0.1474
Epoch 873/1000
11509/11509 [==============================] - 5s 455us/step - loss: 4.2112 - acc: 0.1505 - val_loss: 4.9816 - val_acc: 0.1429
Epoch 874/1000
11509/11509 [==============================] - 5s 453us/step - loss: 4.2190 - acc: 0.1435 - val_loss: 4.9912 - val_acc: 0.1378
Epoch 875/1000
11509/11509 [==============================] - 5s 447us/step - loss: 4.2214 - acc: 0.1433 - val_loss: 4.9859 - val_acc: 0.1431
Epoch 876/1000
11509/11509 [==============================] - 5s 457us/step - loss: 4.2565 - acc: 0.1450 - val_loss: 4.9804 - val_acc: 0.1423
Epoch 877/1000
11509/11509 [==============================] - 5s 450us/step - loss: 4.2171 - acc: 0.1444 - val_loss: 4.9775 - val_acc: 0.1429
Epoch 

Epoch 929/1000
11509/11509 [==============================] - 5s 451us/step - loss: 4.2160 - acc: 0.1468 - val_loss: 4.9770 - val_acc: 0.1421
Epoch 930/1000
11509/11509 [==============================] - 5s 458us/step - loss: 4.2045 - acc: 0.1505 - val_loss: 4.9817 - val_acc: 0.1466
Epoch 931/1000
11509/11509 [==============================] - 5s 452us/step - loss: 4.2021 - acc: 0.1494 - val_loss: 4.9897 - val_acc: 0.1387
Epoch 932/1000
11509/11509 [==============================] - 5s 451us/step - loss: 4.2032 - acc: 0.1478 - val_loss: 5.0076 - val_acc: 0.1376
Epoch 933/1000
11509/11509 [==============================] - 5s 448us/step - loss: 4.2141 - acc: 0.1488 - val_loss: 5.0009 - val_acc: 0.1399
Epoch 934/1000
11509/11509 [==============================] - 5s 466us/step - loss: 4.2086 - acc: 0.1470 - val_loss: 4.9926 - val_acc: 0.1237
Epoch 935/1000
11509/11509 [==============================] - 5s 456us/step - loss: 4.2081 - acc: 0.1461 - val_loss: 4.9793 - val_acc: 0.1421
Epoch 

Epoch 987/1000
11509/11509 [==============================] - 5s 459us/step - loss: 4.2084 - acc: 0.1481 - val_loss: 5.0714 - val_acc: 0.1180
Epoch 988/1000
11509/11509 [==============================] - 5s 458us/step - loss: 4.2502 - acc: 0.1415 - val_loss: 4.9958 - val_acc: 0.1372
Epoch 989/1000
11509/11509 [==============================] - 5s 450us/step - loss: 4.2245 - acc: 0.1477 - val_loss: 4.9991 - val_acc: 0.1376
Epoch 990/1000
11509/11509 [==============================] - 5s 450us/step - loss: 4.2153 - acc: 0.1450 - val_loss: 4.9833 - val_acc: 0.1449
Epoch 991/1000
11509/11509 [==============================] - 5s 455us/step - loss: 4.2092 - acc: 0.1488 - val_loss: 4.9812 - val_acc: 0.1419
Epoch 992/1000
11509/11509 [==============================] - 5s 450us/step - loss: 4.2051 - acc: 0.1488 - val_loss: 4.9900 - val_acc: 0.1376
Epoch 993/1000
11509/11509 [==============================] - 5s 454us/step - loss: 4.2081 - acc: 0.1427 - val_loss: 4.9920 - val_acc: 0.1500
Epoch 

In [128]:
# predict the number of days a person has to stay in the hospital
predictions = model.predict(X_test)

In [129]:
# compare prediction with real
n = 0
print("Predicted: {0} \tcertainty: {1}".format(predictions[n].argmax(axis=0), predictions[n][predictions[n].argmax(axis=0)]))
print("Real: {0}".format(list(y_test)[n]))

Predicted: 13 	certainty: 0.18232372403144836
Real: 20


In [130]:
# make list of all predictions
y_predicted = []
for i in range(0, len(predictions)):
    y_predicted.append(predictions[i].argmax(axis=0))

In [132]:
# convert ints back to ICD codes
icd_test = []
icd_predicted = []
for i in range(0, len(y_test)):
    for key,value in icd9codes_dict.items():
        # convert y_test number to ICD code
        if(value == list(y_test)[i]):
            icd_test.append(key)
        # convert predicted number to ICD code
        elif(value == y_predicted[i]):
            icd_predicted.append(key)

In [133]:
# compare prediction with real
n = 1
print("Predicted: {0}".format(icd_predicted[n]))
print("Real: {0}".format(icd_test[n]))

Predicted: 4A0971Z
Real: 3E0234Z


In [136]:
# save model for later use
# https://stackoverflow.com/questions/40396042/how-to-save-scikit-learn-keras-model-into-a-persistence-file-pickle-hd5-json-ya
model.save('model_procedure_next.h5')
# save ICD code for later use
#https://stackoverflow.com/questions/19201290/how-to-save-a-dictionary-to-a-file
np.save('icd_procedure.npy', icd9codes_dict) 